In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
seed = 18

merged_aspirational_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Aspirational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_aspirational_df = shuffle(merged_aspirational_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_aspirational_df, test_size=0.1, random_state=18, stratify=merged_aspirational_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [20]:
MAXLEN = 150

In [21]:
X = training_df['sentence']
y = training_df['label']

In [22]:
import ktrain
from ktrain import text

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

model_name = 'bert-base-cased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()


preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 57


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 37
	99percentile : 51


In [23]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = {0:0.56103896,1:4.59574468}

[0.57152778 3.99514563]


In [24]:
import tensorflow as tf
import numpy as np
import os
import random
def reset_random_seeds(seed=2):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [25]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
learner.autofit(2e-5, early_stopping=3)
# learner.autofit(2.27E-06, early_stopping=4)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/1024
330/330 [==============================] - 177s 464ms/step - loss: 0.3687 - accuracy: 0.8744 - val_loss: 0.3049 - val_accuracy: 0.8747
Epoch 2/1024
330/330 [==============================] - 117s 352ms/step - loss: 0.2914 - accuracy: 0.8678 - val_loss: 0.3031 - val_accuracy: 0.8717
Epoch 3/1024
330/330 [==============================] - 118s 354ms/step - loss: 0.2210 - accuracy: 0.8987 - val_loss: 0.3126 - val_accuracy: 0.8497
Epoch 4/1024
330/330 [==============================] - ETA: 0s - loss: 0.1955 - accuracy: 0.9170
Epoch 00004: Reducing Max LR on Plateau: new max lr will be 1e-05 (if not early_stopping).
330/330 [==============================] - 117s 353ms/step - loss: 0.1955 - accuracy: 0.9170 - val_loss: 0.3220 - val_accuracy: 0.8390
Epoch 5/1024
330/330 [==============================] - 114s 343ms/step - loss: 0.1297 - accuracy: 

In [26]:
learner.validate(class_names=distillbert_transformer.get_classes())

42/42 [==============================] - 33s 504ms/step
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      1152
           1       0.30      0.02      0.03       165

    accuracy                           0.87      1317
   macro avg       0.59      0.51      0.48      1317
weighted avg       0.80      0.87      0.82      1317



array([[1145,    7],
       [ 162,    3]])

In [27]:
distillbert_predictor = ktrain.get_predictor(learner.model, preproc=distillbert_transformer)

distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

y_pred_distillbert = [int(x) for x in y_pred_distillbert]

tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

True Negative: 1225, False Positive: 11, False Negative: 172, True Positive: 4
  Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.99      0.93      1236
           1       0.27      0.02      0.04       176

    accuracy                           0.87      1412
   macro avg       0.57      0.51      0.49      1412
weighted avg       0.80      0.87      0.82      1412
 



In [15]:
# distillbert_predictor.save('../../../../saved_models/social_bert_base_cased_model_08012024')

: 

In [5]:
learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 29s 427ms/step
              precision    recall  f1-score   support

    negative       0.98      0.95      0.97      1440
    positive       0.71      0.88      0.78       185

    accuracy                           0.94      1625
   macro avg       0.85      0.91      0.88      1625
weighted avg       0.95      0.94      0.95      1625



array([[1373,   67],
       [  23,  162]])